# Reto 03-A-Grounding (Fundamentación) 

## 1. Descripción General 

La Fundamentación (Grounding) es una técnica utilizada cuando deseas que el modelo devuelva respuestas fiables a una pregunta dada. A menudo, los modelos GPT necesitarán contexto adicional para proporcionar una respuesta que no alucine, también conocida como dar respuestas falsas. Recuerda que estos modelos GPT solo han sido entrenados con datos hasta septiembre de 2021. Además, los modelos no han sido entrenados con datos específicos de casos de uso.

Existen un par de métodos para llevar a cabo la fundamentación. En este escenario, nos centraremos principalmente en el grounding básico dentro del prompt. En el reto cuatro, verás otras aplicaciones de fundamentación utilizando una base de conocimientos externa e implementando la técnica de Generación Aumentada con Recuperación, o RAG.

Para entender los conceptos básicos de la fundamentación y sus beneficios, este notebook te guiará a través de un ejemplo. A continuación se muestra el escenario que incorporaremos.

## 2. Escenario

Estás escribiendo un informe sobre el torneo de tenis de Wimbledon y necesitas discutir el último partido. Descubre quién fue el ganador de los individuales masculinos y femeninos en 2023.

## 3. Comencemos con la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos clave de configuración que completaste en las tareas anteriores.

In [1]:
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto de conexión y tus claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [2]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

model=os.getenv("CHAT_MODEL_NAME")

### 3.0 Funciones Auxiliares

**get_completion** ayuda a crear una respuesta de OpenAI utilizando el modelo de completado de texto de tu elección.

In [3]:
def get_chat_completion(prompt, model=model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

### 3.1 Antes de la Fundamentación

#### Tarea #1 del Estudiante:

Edita el prompt en la celda siguiente para hacerle una pregunta al modelo sobre el escenario.

Scenario: You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match. Find out who the 2023 winner was for the mens and womens singles.

In [6]:
prompt = f"""
Scenario: You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match. Find out who the 2023 winner was for the mens and womens singles."""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")


Response: As an AI language model, I cannot provide information about future events as they have not yet occurred. The Wimbledon tennis tournament for 2023 has not taken place yet, and therefore, there are no winners for the men's and women's singles.



¿Es esta la respuesta que esperabas?

### 3.2 Después de la Fundamentación

#### Tarea #2 del Estudiante:

Modifica el prompt a continuación para fundamentar el modelo. ¿Cómo puedes obtener una respuesta más precisa que la que recibiste anteriormente?

In [8]:
text='''Año	Masculino	Femenino
2024
Carlos Alcaraz	Barbora Krejcikova
2023
Carlos Alcaraz	Marketa Vondrousova
2022
N. Djokovic	Elena Rybakina
2021
N. Djokovic	Ashleigh Barty
2020
No se disputó por la pandemia de Covid-19	No se disputó por la pandemia de Covid-19
2019
N. Djokovic	S. Halep
2018
N. Djokovic	A. Kerber
2017
R. Federer	Garbiñe Muguruza
2016
A. Murray	Serena Williams
2015
N. Djokovic	Serena Williams
2014
N. Djokovic	P. Kvitova
2013
A. Murray	M. Bartoli
2012
Roger Federer	Serena Williams
2011
Novak Djokovic	Petra Kvitova
2010
Rafa Nadal	Serena Williams
2009
R. Federer	Serena Williams
2008
Rafa Nadal	Venus Williams
2007
R. Federer	Venus Williams
2006
R. Federer	Amelie Mauresmo
2005
R. Federer	Venus Williams
2004
R. Federer	Maria Sharapova
2003
R. Federer	Serena Williams
2002
L. Hewitt	Serena Williams
2001
G. Ivanisevic	Venus Williams
2000
P. Sampras	Venus Williams
1999
P. Sampras	Lindasay Davenport
1998
P. Sampras	Jana Novotna
1997
P. Sampras	Martina Hingis
1996
R.P.S. Krajicek	Steffi Graff
1995
P. Sampras	Steffi Graff
1994
P. Sampras	Conchita Martínez
1993
P. Sampras	Steffi Graff
1992
A.K. Agassi	Steffi Graff
1991
M.D. Stich	Steffi Graff
1990
S.B. Edberg	Martina Navratilova
1989
B.F. Becker	Steffi Graff
1988
S.B. Edberg	Steffi Graff
1987
P.H. Cash	Martina Navratilova
1986
B.F. Becker	Martina Navratilova
1985
B.F. Becker	Martina Navratilova
1984
J.P. McEnroe	Martina Navratilova
1983
J.P. McEnroe	Martina Navratilova
1982
J.S. Connors	Martina Navratilova
1981
J.P. McEnroe	J.M. Lloyd
1980
B. Borg	R.A. Cawley
1979
B. Borg	Martina Navratilova
1978
B. Borg	Martina Navratilova
1977
B.R. Borg	S.V. Wade
1976
B.R. Borg	C.M. Evert
1975
A.R. Ashe	L.W. King
1974
J.S. Connors	C.M. Evert
1973
J. Kodes	L.W. King
1972
S.R. Smith	L.W. King
1971
J.D. Newcombe	E.F. Goolagong
1970
J.D. Newcombe	B.M. Court
1969
R. Laver	P.F. Jones
1968
R.G. Laver	L.W. King
1967
J.D. Newcombe	L.W. King
1966
M.M. Santana	Billie Jean King
1965
R.S. Emerson	M. Smith
1964
R.S. Emerson	M.E.A. Bueno
1963
C.R. McKinley	M. Smith
1962
R. Laver	J.R. Susman
1961
R.G. Laver	F.A.M. Mortimer
1960
N.A. Fraser	M.E.A. Bueno
1959
A.R. Olmedo	M.E.A. Bueno
1958
A.J. Cooper	A. Gibson
1957
L.A. Hoad	A. Gibson
1956
L.A. Hoad	S.J. Fry
1955
M.A. Trabert	A.L. Brough
1954
J. Drobny	M.C. Connolly
1953
E.V. Seixas	M.C. Connolly
1952
F.A. Sedgman	M.C. Connolly
1951
R. Savitt	D.J. Hart
1950
B. Patty	A.L. Brough
1949
F.R. Schroeder	A.L. Brough
1948
R. Falkenburg	A.L. Brough
1947
J.A. Kramer	M.E. Osborne
1946
Y.F.M. Petra	M.E. Osborne
1945-1940
No se disputó por la II Guerra Mundial	No se disputó por la II Guerra Mundial
1939
R.L. Riggs	A. Marble
1938
J.D. Budge	F.S. Moody
1937
J.D. Budge	D.E. Round
1936
F.J. Perry	H.H. Jacobs
1935
F.J. Perry	F.S. Moody
1934
F.J. Perry	D.E. Round
1933
J.H. Crawford	F.S. Moody
1932
H.E.Vines	F.S. Moody
1931
S.B.B. Wood	C. Aussem
1930
W.T. Tilden	F.S. Moody
1929
H.J. Cochet	H.N. Wills
1928
J.R. Lacoste	H.N. Wills
1927
H.J. Cochet	H.N. Wills
1926
J.R. Borotra	L.A. Godfree
1925
R. Lacoste	S.R.F. Lenglen
1924
J.R. Borotra	K. McKane
1923
W. M. Johnston	S.R.F. Lenglen
1922
G.L. Patterson	S.R.F. Lenglen
1921
W.T. Tilden	S.R.F. Lenglen
1920
W.T. Tilden	S.R.F. Lenglen
1919-1915
No se disputó por la I Guerra Mundial	No se disputó por la I Guerra Mundial
1914
N.E. Brookes	R.L. Chambers
1913
A.F. Wilding	R.L. Chambers
1912
A.F. Wilding	D.T.R. Larcombe (GBR)
1911
A.F. Wilding	R.L. Chambers
1910
A.F. Wilding	R.L. Chambers
1909
A.W. Gore	P.D.H. Boothby
1908
A. Sterry (GBR)	-
1907
M.G. Sutton (EEUU)	-
1906
D.K. Douglass (GBR)	-
1905
M.G. Sutton (EEUU)	-
1904
D.K. Douglass (GBR)	-
1903
D.K. Douglass (GBR)	-
1902
M.E. Robb (GBR)	-
1901
A. Sterry (GBR)	-
1900
G.W. Hillyard (GBR)	-
1899
G.W. Hillyard (GBR)	-
1898
C.R. Cooper (GBR)	-
1897
G.W. Hillyard (GBR)	-
1896
C.R. Cooper (GBR)	-
1895
C.R. Cooper (GBR)	-
1894
G.W. Hillyard (GBR)	-
1893
C. Dod (GBR)	-
1892
C. Dod (GBR)	-
1891
C. Dod (GBR)	-
1890
C. Rice (GBR)	-
1889
G.W. Hillyard (GBR)	-
1888
C. Dod (GBR)	-
1887
C. Dod (GBR)	-
1886
B. Bingley (GBR)	-
1885
M.E.E. Watson (GBR)	-
1884
M.E.E. Watson (GBR)	-'''

In [9]:
prompt = f"""
Scenario: You are writing a report on the Wimbledon tennis tournament and need to discuss the latest match. 
Find out in wikipedia who the 2023 winner was for the mens and womens singles use the text delimited by triple backticks .
```{text}```.
"""

model_response = get_chat_completion(prompt, model=model)
print(f"Response: {model_response}\n")

Response: According to the Wikipedia table, the winner of the 2023 Wimbledon tennis tournament for the men's singles was Carlos Alcaraz, and for the women's singles was Marketa Vondrousova.



¿Es esta la respuesta que esperabas para ayudarte a escribir tu informe?

## Criterios de Éxito

Para completar este desafío con éxito:

* Demuestra que has comprendido cómo fundamentar un modelo y por qué es importante.
* Asegúrate de obtener una respuesta precisa a tu pregunta que te ayudará a completar el escenario descrito al principio del desafío.
